# Movie Recommendation System

### Import all dependencies

In [1]:
import numpy as np
import pandas as pd
import difflib 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data Collection and Preprocessing

In [2]:
# load dataset
movies_data = pd.read_csv("data/movies.csv")

In [3]:
# show dataset
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
# number of rows and columns
movies_data.shape

(4803, 24)

In [5]:
# missing values
movies_data.isnull().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [6]:
# selecting the relevant features for recommendation
selected_features = ["genres", "keywords", "tagline", "cast", "director"]
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [7]:
# replacing null values
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

In [8]:
# combine all the selected features
combined_features = movies_data["genres"]+ " " + movies_data["keywords"] + " " + movies_data["tagline"] + " " + movies_data["cast"] + " " + movies_data["director"]

In [9]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


### Feature Extraction

In [10]:
vectorizer = TfidfVectorizer()

In [11]:
# converting to numerical values
feature_vectors = vectorizer.fit_transform(combined_features)

In [12]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

### Cosine Similarity

In [13]:
similarity = cosine_similarity(feature_vectors)

In [14]:
similarity.shape

(4803, 4803)

In [21]:
print(similarity[0])

[1.         0.07219487 0.037733   ... 0.         0.         0.        ]


In [51]:
movie_name = input("Enter a movie name: ")

Enter a movie name: Spider-Man 3


In [52]:
# creating a list with all the movie names
movie_name_list = movies_data["title"].tolist()

In [53]:
print(movie_name_list)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [54]:
# finding the close match with input
find_close_match = difflib.get_close_matches(movie_name, movie_name_list)

In [55]:
print(find_close_match)

['Spider-Man 3', 'Spider-Man 2', 'Spider-Man']


In [56]:
# the movie name itself
closest_match = find_close_match[0]

In [72]:
# finding index in dataset
index_of_movie = movies_data[movies_data.title == closest_match]["index"].values[0]
index_of_movie

5

In [73]:
# similarity list
similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, 0.07786899789424222), (1, 0.09894823392218867), (2, 0.05423729174306449), (3, 0.045966370176512346), (4, 0.12989685014764749), (5, 1.0000000000000002), (6, 0.008533035265041787), (7, 0.014969430000340272), (8, 0.030672926634831597), (9, 0.02723144740993059), (10, 0.05175228440195727), (11, 0.025246332704586216), (12, 0.028195054162193353), (13, 0.030631948558306837), (14, 0.032615786562290236), (15, 0.02025392118053171), (16, 0.014194201691251133), (17, 0.026971842502583172), (18, 0.011098049208845679), (19, 0.040349767290054524), (20, 0.04114142570617925), (21, 0.012446396087434801), (22, 0.031532104168917156), (23, 0.047576295218824526), (24, 0.053452525356972), (25, 0.013271868195005245), (26, 0.015449753596452911), (27, 0.0482419692051955), (28, 0.02004874136568231), (29, 0.03997257571250261), (30, 0.4903549825429173), (31, 0.024826539308291363), (32, 0.031453427338306174), (33, 0.014876420444054777), (34, 0.0), (35, 0.015616519524535603), (36, 0.014071730318175654), (37, 0.15

In [74]:
# sort this list to have the highest similarity score
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(5, 1.0000000000000002), (30, 0.4903549825429173), (159, 0.3188331558421017), (4395, 0.17888689913112343), (1002, 0.1745470045813942), (37, 0.15554293293271138), (816, 0.15497147906545974), (1523, 0.1401531010856961), (916, 0.13608429870050076), (921, 0.1343632541276121), (1435, 0.12996991847328704), (4, 0.12989685014764749), (4048, 0.128706180314909), (3506, 0.128215335495091), (4675, 0.12655807754026566), (2310, 0.12606919101718606), (2756, 0.12553266396866217), (1975, 0.12551749455381725), (2152, 0.12499864111012884), (857, 0.12476354011486111), (2904, 0.12470032862377442), (2763, 0.1234703406497517), (2121, 0.12335118884862087), (3046, 0.11765442976190879), (1201, 0.11640281561368668), (1191, 0.11554188636113237), (1359, 0.11533013884014888), (210, 0.11438390948446629), (1512, 0.11262454282010023), (1373, 0.11189125300375598), (3479, 0.1117631504176937), (1796, 0.11101582801192564), (3004, 0.1104917820015983), (2369, 0.11026953117345442), (382, 0.10961236428555698), (868, 0.109270

In [38]:
# print similar movies
i = 1
for movie in sorted_similar_movies:
    index = movie[0]
    title = movies_data[movies_data.index == index]["title"].values[0]
    if(i <= 30):
        print(i, ". ", title)
        i += 1

1 .  Avatar
2 .  Alien
3 .  Aliens
4 .  Guardians of the Galaxy
5 .  Star Trek Beyond
6 .  Star Trek Into Darkness
7 .  Galaxy Quest
8 .  Alien³
9 .  Cargo
10 .  Trekkies
11 .  Gravity
12 .  Moonraker
13 .  Jason X
14 .  Pocahontas
15 .  Space Cowboys
16 .  The Helix... Loaded
17 .  Lockout
18 .  Event Horizon
19 .  Space Dogs
20 .  Machete Kills
21 .  Gettysburg
22 .  Clash of the Titans
23 .  Star Wars: Clone Wars: Volume 1
24 .  The Right Stuff
25 .  Terminator Salvation
26 .  The Astronaut's Wife
27 .  Planet of the Apes
28 .  Star Trek
29 .  Wing Commander
30 .  Sunshine


### Recommandation System

In [99]:
# take movie name
movie_name = input("Enter a movie name: ")

# finding the close match with input
find_close_match = difflib.get_close_matches(movie_name, movie_name_list)

# closest match - the searched movie most of the time
closest_match = find_close_match[0]

# index of closest match
index_of_movie = movies_data[movies_data.title == closest_match].values[0][0]

# make a list with similarity socore and index, of that movie
similarity_score = list(enumerate(similarity[index_of_movie]))

# sort to get most similar movies at first
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

# print similar movies
i = 1
for movie in sorted_similar_movies:
    index = movie[0]
    title = movies_data[movies_data.index == index]["title"].values[0]
    if(i <= 10):
        print(i, ". ", title)
        i += 1


Enter a movie name: iron man
1 .  Iron Man
2 .  Iron Man 2
3 .  Iron Man 3
4 .  Avengers: Age of Ultron
5 .  The Avengers
6 .  Captain America: Civil War
7 .  Captain America: The Winter Soldier
8 .  Ant-Man
9 .  X-Men
10 .  Made
